# CoDaS-HEP Columnar Data Analysis, part 1

## Programming paradigms

### Imperative

### Functional

### Object-oriented

### Array-oriented

### Declarative

### ~~Which is best?~~ Which problems are each best suited for?

## Array-oriented programming

### History

### Synergy with data analysis

### Array-oriented in Python: NumPy and everything else

### Limitations of array-oriented programming

### Accelerating code in Python

## From ROOT files into arrays

### Uproot

### Awkward Array

## Project: H → ZZ → 4ℓ

### 4 leptons of the same flavor

### Opposite charges

### On your own: the H → ZZ → 2μ2e case

### Hint!